In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import treelib


# Table of contents:
* [What are Decision trees](#1)
* [Overview of the algorithm](#2)
* [Decision Trees Superpowers](#3)
* [Decision Trees are Effective when](#4)
* [When to watch out for?](#5)
* [Code](#6)
* [Node splitting metrics](#7)
* [Decision tree class](#8)


# What are Decision trees? <a class="anchor" id="1"></a>

Decision trees are a popular supervised learning algorithm used for **classification** and **regression** tasks. <br>
Although, relatively less asked in interviews, it is one of the most versatile method when it comes to application. <br>
It is like a swiss knife of tools and its derivatives such as random forests in combination with techniques like bagging, boosting etc can make it really effective.

# Overview of the algorithm <a class="anchor" id="2"></a>

1. Data preprocessing: <br>
    In general we **don't need to normalize** data, because decision tree is invariant to scale. However, if your data is high dimensional, using PCA to remove un-neccesary features may be a good idea. <br>
    
2. Find the best node to split: <br>
    This is the greedy part. Suppose you had a metric to measure if your data is more or less structured at a split level. Let's call this metric $M$. If M is high, there is more randomness, less structure. <br>
    
    *We start from the root node, where we have all the data* <br>
    $M_P =   M(root) $ which is the randomness in the root level
    Now, we do the following:
    * for each feature in data $F$:
        * for each value in given data of that feature $V$:
            We split the the data into two parts: <br>
                Left part: All the samples where $F_i <= V$ and, <br>
                Right part: All the samples where $F_i > V$ and, <br>
                $M_L$ is metric of randomness for left and $M_R$ is for right child, <br>
                and we are interested in this quantity: <br>
                
                $Gain = M_P - (M_L + M_R)$ <br> <br>
                
    
    *Save the feature $F$ and the value of that feature $V$ which gives you the maximum Gain.*: This means the current split, brings the most order in the dataset. <br>
    If the difference is high, it means we went from high disorder to more structure.
    
3. Keep doing step 2 recursively for all the left and right nodes thus created in the process till one of two happens:
    *  Data in the split has all the samples of one class, or <br>
    *  We have reached the max-depth in the tree (a user-defined hyperparam)
    
            
    


# Decision Trees Superpowers <a class="anchor" id="3"></a>

Here are some Decision Trees superpowers: <br>

 *  *Decision Trees don't make assumptions about the underlying data distribution:* <br>
     They can handle linear as well as non linear data. Thsi property is also called non-parametric property of Decision trees.
 *  *Scale invariant*: <br>
     Decision trees are invariant to the scale of the data. So, we don't have to normalize data.
 *  *Mixed data handling*: <br>
     Decision trees can handle both categorical and numerical features. They can split the data based on categorical variables and use numerical thresholds for continuous variables.
 *  *Robust to Outliers*:
     Since, the decision tree algorithms work by creating binary splits, any outlier case will likely form a split with only itself in it and will not impact the overall performance.
 *  *Can understand Feature importance*: <br>
     By examining how frequently a feature is used for splitting, we can determine its relative importance in the decision-making process. This is a very distiguishing aspect of Decision Trees.
 *  *Interpretable*: <br>
     Decision trees splits can be visualized and explained unlike other algorithms where explainability is an issue (ex. Neural networks).
 *  *Handles Missing data*: <br>
     Decision trees can handle missing values by using surrogate splits. Surrogate splits allow decision trees to consider alternative features in case a specific feature has missing data.
     
     
     
What all this means is Decision trees are very versatile. <br>
If you're asked a simple algorithms which can classify a 1-D data where you don't know the underlying distribution, Decision Trees are a very efficient solution. <br>


# Decision Trees are Effective when: <a class="anchor" id="4"></a>

 *  Features have non-linear relationships.
 *  Features have interactions with certain other featues.
 *  Most importantly, if the dataset is such that there can be discrete decisino boundaries, decision trees are extremely effective.

     
    
        
    


# When to watch out for? <a class="anchor" id="5"></a>

 *  *High Dimensional data:* <br>
     Whenever there are high number of features, there is a requirement for higher samples as well to understand complex decision boundaries. <br>
     In such a case, decision tree may not be the best solution. As a rough guideline, having at least 5 to 10 times as many samples as features is often considered a reasonable starting point. 
 *  *Prone to overfitting if not enough samples:* <br>
     Since decision tree is a greedy splitting algorithm, in absence of data, it will overfit to however much data it has, leading to poor generalization. 
 *  *Data with complex dependencies:* <br>
     When features have complex non linear dependencies, a decision may not yield good results. Say, more than 3 features have spectral relationships. In such a case, we need to use other techniques such as random forest, gradient boosting.
 *  *Imbalanced classes:* <br>
     Just like many other algorithms (like logistic regression etc), we need to be careful about class imbalance. However, this issue can be mitigated if we're careful. We can use class weighting or resampling.
     
     

# Coding it <a class="anchor" id="6"></a>

## 1. Load the dataset

In [2]:
rootpath = "../../"

In [3]:
csvpath = os.path.join(rootpath, "assets/iris.csv")
df = pd.read_csv(csvpath)
num_samples, num_feats = df.shape
print(f"Total samples: {num_samples}")
print(f"Total features: {num_feats}")
feat_names = list(df)
print(f"Features: {feat_names}")


Total samples: 150
Total features: 5
Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']


In [4]:
## Three class labels
df.species.value_counts()

species
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64

#### 1.1 Create Train and Test Splits

In [5]:
random_state = 42 # this is the answer. But what is the question?
df_random = df.sample(frac=1, random_state=random_state)

dftrain = df_random[0:120]
dftest = df_random[120:]
dftrain.shape, dftest.shape

((120, 5), (30, 5))

## 2. Define helper functions <a class="anchor" id="7"></a>

#### 2.1 Gini Inpurity

Suppose we have a vector $L$ of labels (integers or chars), 
we can count the occurence of each label (get their frequency). <br>
Subsequently, the probability for each label $x$ is defined as $P(x_i)$
then the GINI inpurity is defined as: <br>
    $$GINI(x) = 1 - \sum_i^N (P(x_i))^2$$
    

In [6]:
def gini_impurity(x):
    unq, counts = np.unique(x, return_counts=True)
    probs = counts / len(x)
    gini = 1 - (probs **2 ).sum()
    return gini



#### 2.2 Entropy

Suppose we have a vector $L$ of labels (integers or chars), 
we can count the occurence of each label (get their frequency). <br>
Subsequently, the probability for each label $x$ is defined as $P(x_i)$
then the Entropy is defined as: 
    $$Entropy(x) = \sum_i^N -P(x_i)\log_2 (P(x_i))$$
    

In [7]:
def entropy(x):
    unq, counts = np.unique(x, return_counts=True)
    probs = counts/len(x)
    entropy = -probs * np.log2(probs)
    entropy = entropy.sum()
    return entropy



#### 2.3 Information Gain



In [8]:
def information_gain(feats: np.ndarray,
                     labels: np.ndarray,
                     value: float,
                     metric='gini'):
    assert len(feats) == len(labels)  , f"len(feats): {len(feats)} | len(labels): {len(labels)}"  
    f = gini_impurity if metric == "gini" else entropy
    left_mask = feats <= value
    left_y = labels[left_mask]
    right_y = labels[~left_mask]
    
    left_wt = len(left_y) / len(labels)
    right_wt = len(right_y) / len(labels)
    
    gain = f(labels) - ( left_wt * f(left_y) +  right_wt * f(right_y) )
    return gain



## 3. Combine them in a class <a class="anchor" id="8"></a>

In [9]:
class Node:
    # class-level variable to automatically assign the ID of each node
    ID_CTR = 0
    def __init__(self):
        self.id = Node.ID_CTR
        Node.ID_CTR+=1
        self.level = -1
        self.split_feature = None
        self.split_value = None
        
        ## If its a leaf node
        self.isLeaf = True
        self.leaf_label = None
        
        # Children
        self.left = None
        self.right = None
    
    @classmethod
    def make_split_node(cls, level, split_feature, split_value):
        # Split node contain the information about the feature F and the value of F: V, which is used to split
        node = Node()
        node.level = level
        node.split_feature = split_feature
        node.split_value = split_value
        node.isLeaf = False
        return node
    
    @classmethod
    def make_leaf_node(cls, level, label):
        # Leaf node only contain the information of the label
        node = Node()
        node.isLeaf = True
        node.level = level
        node.leaf_label = label
        return node
    
    def __repr__(self):
        s = f"L: {self.level}"
        if self.isLeaf:
            s+= f"| Label: {self.leaf_label}"
            return s
        
        s += f"| Feat: {self.split_feature}"
        s += f"| Thr: {self.split_value}"
        return s
        

class DecisionTree:
    
    def __init__(self, max_depth: int):
        self.root: Node = None
        self.max_depth: int = max_depth
        from treelib import Tree, Node
        self.prtree = Tree()
        
    def fit_tree(self, X, Y):
        X = np.array(X)
        Y = np.array(Y)
        assert len(X) == len(Y), f"Labels and Data sizes are not same: X:{len(X)}, Y: {len(Y)}"
        self.root = self._build_tree(X, Y, 0)
        
    def _build_tree(self, X, Y, level):
        """
        1. Check if termination is reached because all labels are same 
           or max depth is reached
        2. If Max Depth is reached, then we return the label as the max count in the mixture.
        """
        if level == self.max_depth or len(np.unique(Y)) == 1:
            # terminate
            if len(np.unique(Y)) == 1: # If there is only one type of samples in the split.
                node = Node.make_leaf_node(level, np.unique(Y)[0])
            elif len(np.unique(Y)) > 1: # We have max depth, so we will return the majority sample in this split
                labels, counts = np.unique(Y, return_counts=True)
                node = Node.make_leaf_node(level, labels[counts.argmax()])
            else:
                print("failure: ", level, Y)
            
            return node
            
        best_split = self._find_best_split(X, Y)
        root = Node.make_split_node(level, 
                                    best_split["feature_id"], 
                                    best_split["value"])
        
        
        f = best_split["feature_id"]
        v = best_split["value"]
        left_mask = X[:, f] <= v
        if left_mask.sum() > 0:
            root.left = self._build_tree(X[left_mask], Y[left_mask], level + 1)
        if (~left_mask).sum() > 0:
            root.right = self._build_tree(X[~left_mask], Y[~left_mask], level + 1)
        return root
        
    def _find_best_split(self, X, Y):
        """
        Goes through each value in each feature and computes the gain. 
        Returns the best split corresponding to max gain
        """
        max_gain = float('-inf')
        best_split = {}

        for f in range(X.shape[1]):
            feats = X[:, f]
            unq_values = np.unique(feats)
            for value in unq_values:
                gain = information_gain(feats, Y, value, metric="gini")
                if gain > max_gain:
                    best_split["feature_id"] = f
                    best_split["value"] = value
                    best_split["gain"] = gain
                    max_gain = gain
        return best_split
    
    def traverse_tree(self, X):
        cur = self.root
        while cur.isLeaf is False:
            if X[cur.split_feature] <= cur.split_value:
                cur = cur.left
            else:
                cur = cur.right
        return cur.leaf_label
    
    def predict(self, X):
        preds = []
        for row in X:
            preds.append(self.traverse_tree(row))
        return preds
    
    
    def _print_tree_helper(self, root, parent):
        if root is None:
            return
        self.prtree.create_node(str(root), root, parent=parent)
        if root.isLeaf:
            return
        self._print_tree_helper(root.left, root)
        self._print_tree_helper(root.right, root)
        
            
    def print_tree(self):
        root = self.root
        self.prtree.create_node("ROOT", "root")
        self._print_tree_helper(self.root, "root")
        self.prtree.show()

        

In [10]:
dtree = DecisionTree(3)
dtree.fit_tree(dftrain.drop("species", axis=1).to_numpy(), 
               dftrain["species"].to_numpy())


## 3.1 Visualization of tree

In [11]:
dtree.print_tree()

ROOT
└── L: 0| Feat: 2| Thr: 1.9
    ├── L: 1| Feat: 3| Thr: 1.7
    │   ├── L: 2| Feat: 2| Thr: 4.8
    │   │   ├── L: 3| Label: virginica
    │   │   └── L: 3| Label: virginica
    │   └── L: 2| Feat: 2| Thr: 4.9
    │       ├── L: 3| Label: versicolor
    │       └── L: 3| Label: virginica
    └── L: 1| Label: setosa



In [12]:
xtest = dftest.drop("species", axis=1).to_numpy()
print(xtest.shape)
preds = dtree.predict(xtest)

(30, 4)


## 3.2 Finding accuracy 

In [13]:
gt = dftest.species
TP = preds == gt
accuracy = TP.to_numpy().sum() / len(xtest)
print(f"Accuracy on the test set: {100*np.round(accuracy, 4)}%") 

Accuracy on the test set: 96.67%
